In [ ]:
import os
from dotenv import load_dotenv

from oci.auth.signers import InstancePrincipalsSecurityTokenSigner
from oci.generative_ai.generative_ai_client import GenerativeAiClient
from oci.generative_ai_inference.generative_ai_inference_client import GenerativeAiInferenceClient
from oci.generative_ai_inference.models import (
    ChatDetails,
    OnDemandServingMode,
    CohereChatRequest, 
    CohereTool, 
    CohereParameterDefinition,
    CohereToolCall,
    CohereToolResult
)

In [ ]:
load_dotenv()
compartment_id = os.getenv("COMPARTMENT_ID")
i_endpoint = os.getenv("INFERENCE_ENDPOINT")
signer = InstancePrincipalsSecurityTokenSigner()
generative_ai_inference_client = GenerativeAiInferenceClient(config = {}, signer = signer, service_endpoint = i_endpoint)

In [ ]:
response = generative_ai_inference_client.chat(
    chat_details=ChatDetails(
        compartment_id=compartment_id,
        serving_mode=OnDemandServingMode(
            model_id="cohere.command-r-16k"
        ),
        chat_request=CohereChatRequest(
            message="107-0061の郵便番号の住所を教えてください",
            max_tokens=200,
        )
    )
)

print(response.data)

In [ ]:
import requests
import re

def query_address_from_postal_code(postal_code: str) -> dict:
    """
    Function to retrieve the address for the given postal code
    """
    if re.match("^[0-9]{3}-[0-9]{4}$", postal_code):
        postal_code = postal_code.replace("-", "")
    headers = {"content-type": "application/json"}
    response = requests.get(f"https://jp-postal-code-api.ttskch.com/api/v1/{postal_code}.json", headers=headers)
    data_ja = response.json()["addresses"][0]["ja"]
    if response.status_code == 200:
        return {
            "address": f"{data_ja['prefecture']}{data_ja['address1']}{data_ja['address2']}"
        }
    else:
        return {
            "address": "与えられた郵便番号に合致する住所は見つかりませんでした"
        }

In [ ]:
print(query_address_from_postal_code("107-0061"))

In [ ]:
tools = [
    CohereTool(
        name = "query_address_from_postal_code",
        description = "郵便番号を元に住所を検索します",
        parameter_definitions = {
            "postal_code": CohereParameterDefinition(
                description = "この郵便番号を元に住所を検索します。",
                type = "str",
                is_required = True
            )
        }
    )
]

In [ ]:
response = generative_ai_inference_client.chat(
    chat_details=ChatDetails(
        compartment_id=compartment_id,
        serving_mode=OnDemandServingMode(
            model_id="cohere.command-r-16k"
        ),
        chat_request=CohereChatRequest(
            message="107-0061の郵便番号の住所を教えてください",
            chat_history=response.data.chat_response.chat_history,
            max_tokens=200,
            tools=tools
        )
    )
)

print(response.data)

In [ ]:
functions_map = {
    "query_address_from_postal_code": query_address_from_postal_code
}

In [ ]:
tool_results = []
for tool_call in response.data.chat_response.tool_calls:
    output = functions_map[tool_call.name](**tool_call.parameters)
    outputs = [output]
    tool_results.append(
        CohereToolResult(
            call=CohereToolCall(
                name=tool_call.name,
                parameters=tool_call.parameters
            ),
            outputs=outputs
        )
    )

print(tool_results)

In [ ]:
preamble_override = """
## タスクとコンテキスト
人々の質問やその他のリクエストにインタラクティブに答える手助けをする。あらゆる種類のトピックについて、非常に幅広い質問を受けます。幅広い検索エンジンや類似のツールが用意されており、それらを使って答えを調べます。あなたは、ユーザーのニーズにできる限り応えることに集中すべきです。

## スタイルガイド
ユーザーから別の回答スタイルを要求されない限り、適切な文法とスペルを使い、完全な文章で回答してください。
"""

response = generative_ai_inference_client.chat(
    chat_details=ChatDetails(
        compartment_id=compartment_id,
        serving_mode=OnDemandServingMode(
            model_id="cohere.command-r-16k"
        ),
        chat_request=CohereChatRequest(
            message="107-0061の住所を教えてください。",
            chat_history=response.data.chat_response.chat_history,
            max_tokens=200,
            is_force_single_step=True,
            tools=tools,
            tool_results=tool_results,
            preamble_override=preamble_override
        )
    )
)

print(response.data.chat_response.text)